 # BRAIN WALL
 

In [ ]:
"""
Hole in the Wall - Juego de Poses con Visión por Computador (ULTRA MEJORADO)
Requisitos: pip install opencv-python mediapipe pygame numpy
"""

import cv2
import mediapipe as mp
import pygame
import numpy as np
import random
import time

# Configuración
WINDOW_WIDTH = 1280
WINDOW_HEIGHT = 720
FPS = 30

# Colores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 150, 255)
GREEN = (100, 255, 100)
RED = (255, 100, 100)
YELLOW = (255, 255, 100)
PURPLE = (180, 100, 255)
ORANGE = (255, 165, 0)

# Poses SIMPLIFICADAS - Solo verificamos puntos clave importantes
POSES = [
    {
        "name": "T-Pose",
        "description": "Brazos extendidos horizontalmente",
        "difficulty": "easy",
        "check_points": {
            # Solo verificamos que los brazos estén horizontales
            "left_arm_up": False,
            "right_arm_up": False,
            "left_arm_side": True,
            "right_arm_side": True,
        }
    },
    {
        "name": "Pierna Izq Arriba",
        "description": "Levanta la pierna izquierda",
        "difficulty": "medium",
        "check_points": {
            "left_arm_up": False,
            "right_arm_up": False,
            "left_arm_side": False,
            "right_arm_side": False,
            "left_leg_up": True,
            "right_leg_up": False,
            "left_leg_side": False,
            "right_leg_side": False,
        }
    },
    {
        "name": "Brazos Arriba",
        "description": "Levanta ambos brazos",
        "difficulty": "easy",
        "check_points": {
            "left_arm_up": True,
            "right_arm_up": True,
            "left_arm_side": False,
            "right_arm_side": False,
        }
    },
    {
        "name": "Brazos Abajo",
        "description": "Brazos pegados al cuerpo",
        "difficulty": "easy",
        "check_points": {
            "left_arm_up": False,
            "right_arm_up": False,
            "left_arm_side": False,
            "right_arm_side": False,
        }
    },
    {
        "name": "Brazo Izq Arriba",
        "description": "Solo brazo izquierdo arriba",
        "difficulty": "medium",
        "check_points": {
            "left_arm_up": True,
            "right_arm_up": False,
            "left_arm_side": False,
            "right_arm_side": False,
        }
    },
    {
        "name": "Brazo Der Arriba",
        "description": "Solo brazo derecho arriba",
        "difficulty": "medium",
        "check_points": {
            "left_arm_up": False,
            "right_arm_up": True,
            "left_arm_side": False,
            "right_arm_side": False,
        }
    },
]

# Conexiones del cuerpo para dibujar
BODY_CONNECTIONS = [
    (11, 12),  # hombros
    (11, 13), (13, 15),  # brazo izq
    (12, 14), (14, 16),  # brazo der
    (11, 23), (12, 24),  # torso
    (23, 24),  # caderas
    (23, 25), (25, 27),  # pierna izq
    (24, 26), (26, 28),  # pierna der
]


class HoleInTheWallGame:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Hole in the Wall - ¡Haz la Pose!")
        self.screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        self.clock = pygame.time.Clock()
        
        # MediaPipe
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5,
            model_complexity=1
        )
        # MediaPipe Selfie Segmentation
        self.mp_seg = mp.solutions.selfie_segmentation
        self.segmenter = self.mp_seg.SelfieSegmentation(model_selection=1)
        self.segmented_surface = None
        
        # Cámara
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        # Estado del juego
        self.score = 0
        self.lives = 3
        self.level = 1
        self.current_pose = None
        self.wall_position = WINDOW_WIDTH
        self.wall_speed = 1.5  # MUY LENTO
        self.time_per_wall = 10.0  # MUCHO TIEMPO
        self.wall_start_time = time.time()
        self.game_state = "menu"
        self.last_match_score = 0
        self.show_result = False
        self.result_time = 0
        self.countdown = 3
        self.countdown_start = 0
        self.showing_countdown = False
        
        # Landmarks detectados
        self.detected_landmarks = {}
        
        # Histórico de scores para suavizar
        self.score_history = []
        self.max_history = 15
        
        # Fuentes
        self.font_large = pygame.font.Font(None, 72)
        self.font_medium = pygame.font.Font(None, 48)
        self.font_small = pygame.font.Font(None, 36)
        
        self.select_new_pose()
    
    def select_new_pose(self):
        # Seleccionar pose según nivel
        if self.level <= 3:
            available = [p for p in POSES if p.get("difficulty") == "easy"]
        else:
            available = POSES
        
        self.current_pose = random.choice(available if available else POSES)
        self.wall_position = WINDOW_WIDTH
        self.wall_start_time = time.time()
        self.show_result = False
        self.showing_countdown = True
        self.countdown_start = time.time()
        self.countdown = 3
        self.score_history = []
    
    def process_camera(self):
        ret, frame = self.cap.read()
        if not ret:
            return None
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Obtener pose
        results = self.pose.process(rgb_frame)

        self.detected_landmarks = {}
        if results.pose_landmarks:
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                if landmark.visibility > 0.5:
                    self.detected_landmarks[idx] = (landmark.x, landmark.y, landmark.z)

        # Segmentación del jugador
        seg_results = self.segmenter.process(rgb_frame)
        mask = seg_results.segmentation_mask

        mask_3ch = np.stack((mask, mask, mask), axis=-1)
        person = (mask_3ch > 0.4) * rgb_frame

        person_rgba = np.zeros((person.shape[0], person.shape[1], 4), dtype=np.uint8)
        person_rgba[:, :, :3] = person
        person_rgba[:, :, 3] = (mask * 255).astype(np.uint8)

        person_surface = pygame.image.frombuffer(
            person_rgba.flatten(), (person_rgba.shape[1], person_rgba.shape[0]), "RGBA"
        )

        person_surface = pygame.transform.scale(person_surface, (480, 360))
        self.segmented_surface = person_surface

        return person_surface
    
    def check_leg_position(self, hip_idx, knee_idx, ankle_idx):
        """
        Determina si una pierna está arriba, recta, al lado o abajo.
        Retorna: 'up', 'side', 'straight', 'down'
        """
        if hip_idx not in self.detected_landmarks:
            return None
        if knee_idx not in self.detected_landmarks:
            return None
        if ankle_idx not in self.detected_landmarks:
            return None

        hip = self.detected_landmarks[hip_idx]
        knee = self.detected_landmarks[knee_idx]
        ankle = self.detected_landmarks[ankle_idx]

        # Pierna levantada (rodilla más alta que la cadera)
        if knee[1] < hip[1] - 0.12:
            return 'up'

        # Pierna al lado (pie muy desplazado lateralmente)
        if abs(ankle[0] - hip[0]) > 0.20:
            return 'side'

        # Pierna recta (vertical)
        if abs(knee[0] - hip[0]) < 0.10:
            return 'straight'

        # Si nada aplica, está abajo
        return 'down'


    def check_arm_position(self, shoulder_idx, elbow_idx, wrist_idx):
        """
        Determina si un brazo está arriba, al lado, o abajo
        Retorna: ('up', 'side', 'down')
        """
        if shoulder_idx not in self.detected_landmarks:
            return None
        if elbow_idx not in self.detected_landmarks:
            return None
        if wrist_idx not in self.detected_landmarks:
            return None
        
        shoulder = self.detected_landmarks[shoulder_idx]
        elbow = self.detected_landmarks[elbow_idx]
        wrist = self.detected_landmarks[wrist_idx]
        
        # Verificar si está arriba (muñeca más alta que hombro)
        if wrist[1] < shoulder[1] - 0.15:  # MUY permisivo
            return 'up'
        
        # Verificar si está al lado (muñeca a la misma altura aprox)
        if abs(wrist[1] - shoulder[1]) < 0.2:  # MUY permisivo
            # Y además debe estar alejada lateralmente
            if abs(wrist[0] - shoulder[0]) > 0.15:
                return 'side'
        
        # Si no, está abajo
        return 'down'
    
    def calculate_match_score(self):
        if not self.current_pose or not self.detected_landmarks:
            return 0

        # Necesitamos brazos y piernas
        required = [11,12,13,14,15,16, 23,24,25,26,27,28]
        if not all(p in self.detected_landmarks for p in required):
            return 0

        target = self.current_pose["check_points"]

        # Obtener posiciones reales
        left_arm = self.check_arm_position(11, 13, 15)
        right_arm = self.check_arm_position(12, 14, 16)
        
        left_leg = self.check_leg_position(23, 25, 27)
        right_leg = self.check_leg_position(24, 26, 28)

        points = 0
        max_points = 8  # 4 brazos + 4 piernas

        # --- Brazos ---
        if target.get("left_arm_up") == (left_arm == "up"): points += 1
        if target.get("right_arm_up") == (right_arm == "up"): points += 1
        if target.get("left_arm_side") == (left_arm == "side"): points += 1
        if target.get("right_arm_side") == (right_arm == "side"): points += 1

        # --- Piernas ---
        if target.get("left_leg_up") == (left_leg == "up"): points += 1
        if target.get("right_leg_up") == (right_leg == "up"): points += 1
        if target.get("left_leg_side") == (left_leg == "side"): points += 1
        if target.get("right_leg_side") == (right_leg == "side"): points += 1

        return int((points / max_points) * 100)

    def get_smoothed_score(self):
        """Suavizar el score para evitar fluctuaciones"""
        current = self.calculate_match_score()
        self.score_history.append(current)
        
        if len(self.score_history) > self.max_history:
            self.score_history.pop(0)
        
        return int(sum(self.score_history) / len(self.score_history))
    
    def draw_pose_indicator(self):
        """Dibuja una figura simple mostrando la pose objetivo"""
        if not self.current_pose:
            return
        
        # Posición del indicador
        ind_x = WINDOW_WIDTH - 250
        ind_y = 150
        size = 150
        
        # Fondo
        pygame.draw.rect(self.screen, (40, 40, 60), (ind_x - 20, ind_y - 20, size + 40, size + 60))
        pygame.draw.rect(self.screen, WHITE, (ind_x - 20, ind_y - 20, size + 40, size + 60), 2)
        
        # Título
        title = self.font_small.render("Pose Objetivo:", True, YELLOW)
        self.screen.blit(title, (ind_x - 10, ind_y - 50))
        
        # Dibujar figura simple
        center_x = ind_x + size // 2
        center_y = ind_y + size // 2
        
        # Cabeza
        pygame.draw.circle(self.screen, WHITE, (center_x, center_y - 40), 20)
        
        # Cuerpo
        pygame.draw.line(self.screen, WHITE, (center_x, center_y - 20), (center_x, center_y + 40), 4)
        
        target = self.current_pose["check_points"]
        
        # Brazo izquierdo
        if target["left_arm_up"]:
            # Arriba
            pygame.draw.line(self.screen, GREEN, (center_x, center_y - 10), (center_x - 40, center_y - 50), 4)
        elif target["left_arm_side"]:
            # Al lado
            pygame.draw.line(self.screen, GREEN, (center_x, center_y - 10), (center_x - 60, center_y - 10), 4)
        else:
            # Abajo
            pygame.draw.line(self.screen, GREEN, (center_x, center_y - 10), (center_x - 30, center_y + 30), 4)
        
        # Brazo derecho
        if target["right_arm_up"]:
            # Arriba
            pygame.draw.line(self.screen, GREEN, (center_x, center_y - 10), (center_x + 40, center_y - 50), 4)
        elif target["right_arm_side"]:
            # Al lado
            pygame.draw.line(self.screen, GREEN, (center_x, center_y - 10), (center_x + 60, center_y - 10), 4)
        else:
            # Abajo
            pygame.draw.line(self.screen, GREEN, (center_x, center_y - 10), (center_x + 30, center_y + 30), 4)
        
        # Piernas simples
        pygame.draw.line(self.screen, WHITE, (center_x, center_y + 40), (center_x - 20, center_y + 80), 4)
        pygame.draw.line(self.screen, WHITE, (center_x, center_y + 40), (center_x + 20, center_y + 80), 4)
        
        # Estado actual de los brazos
        left_pos = self.check_arm_position(11, 13, 15)
        right_pos = self.check_arm_position(12, 14, 16)
        
        status_y = ind_y + size + 20
        
        if left_pos:
            left_text = self.font_small.render(f"Izq: {left_pos}", True, WHITE)
            self.screen.blit(left_text, (ind_x, status_y))
        
        if right_pos:
            right_text = self.font_small.render(f"Der: {right_pos}", True, WHITE)
            self.screen.blit(right_text, (ind_x, status_y + 35))

    def draw_player_skeleton(self):
        """Dibujar jugador con indicadores de acierto"""
        if self.segmented_surface:
            self.screen.blit(self.segmented_surface, (50, 110))

        if not self.detected_landmarks:
            return

        player_x = 50
        player_y = 110
        w = 480
        h = 360

        # Dibujar conexiones
        for a, b in BODY_CONNECTIONS:
            if a in self.detected_landmarks and b in self.detected_landmarks:
                x1 = player_x + int(self.detected_landmarks[a][0] * w)
                y1 = player_y + int(self.detected_landmarks[a][1] * h)
                x2 = player_x + int(self.detected_landmarks[b][0] * w)
                y2 = player_y + int(self.detected_landmarks[b][1] * h)
                
                pygame.draw.line(self.screen, GREEN, (x1, y1), (x2, y2), 6)

        # Dibujar puntos importantes más grandes
        important_points = [11, 12, 13, 14, 15, 16]  # hombros, codos, muñecas
        
        for idx, (x, y, z) in self.detected_landmarks.items():
            px = player_x + int(x * w)
            py = player_y + int(y * h)
            
            if idx in important_points:
                pygame.draw.circle(self.screen, YELLOW, (px, py), 12)
                pygame.draw.circle(self.screen, GREEN, (px, py), 10)
            else:
                pygame.draw.circle(self.screen, (200, 200, 200), (px, py), 8)

    def draw_ui(self):
        # Puntuación
        score_text = self.font_medium.render(f"Puntos: {self.score}", True, WHITE)
        self.screen.blit(score_text, (20, 20))
        
        # Vidas
        lives_text = self.font_medium.render(f"Vidas: {'❤️ ' * self.lives}", True, RED)
        self.screen.blit(lives_text, (20, 70))
        
        # Nivel
        level_text = self.font_small.render(f"Nivel: {self.level}", True, WHITE)
        self.screen.blit(level_text, (20, 120))
        
        # Nombre de la pose
        if self.current_pose:
            pose_text = self.font_medium.render(
                f"{self.current_pose['name']}", True, YELLOW)
            self.screen.blit(pose_text, (WINDOW_WIDTH//2 - 120, 20))
            
            desc_text = self.font_small.render(
                self.current_pose['description'], True, WHITE)
            self.screen.blit(desc_text, (WINDOW_WIDTH//2 - 140, 65))
        
        # Match score GRANDE y VISIBLE
        match = self.get_smoothed_score()
        if match >= 75:
            color = GREEN
            status = "¡PERFECTO! ✓"
        elif match >= 50:
            color = YELLOW
            status = "¡BIEN! Mantén"
        elif match >= 25:
            color = ORANGE
            status = "Casi... Ajusta"
        else:
            color = RED
            status = "Revisa la pose"
        
        # Score grande
        match_text = self.font_large.render(f"{match}%", True, color)
        self.screen.blit(match_text, (WINDOW_WIDTH//2 - 50, WINDOW_HEIGHT - 120))
        
        status_text = self.font_medium.render(status, True, color)
        self.screen.blit(status_text, (WINDOW_WIDTH//2 - 120, WINDOW_HEIGHT - 70))
        
        # Barra de tiempo
        elapsed = time.time() - self.wall_start_time
        remaining = max(0, self.time_per_wall - elapsed)
        bar_width = int((remaining / self.time_per_wall) * 400)
        
        bar_x = WINDOW_WIDTH//2 - 200
        bar_y = WINDOW_HEIGHT - 30
        
        pygame.draw.rect(self.screen, (50, 50, 50), (bar_x, bar_y, 400, 20))
        
        if remaining > 5:
            bar_color = GREEN
        elif remaining > 2:
            bar_color = YELLOW
        else:
            bar_color = RED
            
        pygame.draw.rect(self.screen, bar_color, (bar_x, bar_y, bar_width, 20))
        
        time_text = self.font_small.render(f"{remaining:.1f}s", True, WHITE)
        self.screen.blit(time_text, (WINDOW_WIDTH//2 - 30, bar_y - 2))
        
        # Resultado temporal
        if self.show_result and time.time() - self.result_time < 2.5:
            if self.last_match_score >= 40:  # Umbral MUY bajo
                result_text = self.font_large.render(f"¡PASASTE! +{self.last_match_score * 2}", True, GREEN)
            else:
                result_text = self.font_large.render(f"¡INTENTA DE NUEVO! ({self.last_match_score}%)", True, RED)
            
            text_rect = result_text.get_rect(center=(WINDOW_WIDTH//2, WINDOW_HEIGHT//2 - 50))
            
            # Fondo semitransparente
            s = pygame.Surface((text_rect.width + 60, text_rect.height + 40))
            s.set_alpha(220)
            s.fill((0, 0, 0))
            self.screen.blit(s, (text_rect.x - 30, text_rect.y - 20))
            
            self.screen.blit(result_text, text_rect)
        
        # Countdown inicial
        if self.showing_countdown:
            elapsed = time.time() - self.countdown_start
            remaining = 3 - int(elapsed)
            
            if remaining > 0:
                countdown_text = self.font_large.render(str(remaining), True, YELLOW)
                text_rect = countdown_text.get_rect(center=(WINDOW_WIDTH//2, WINDOW_HEIGHT//2 - 100))
                
                # Fondo
                s = pygame.Surface((150, 150))
                s.set_alpha(200)
                s.fill((0, 0, 0))
                self.screen.blit(s, (text_rect.x - 40, text_rect.y - 40))
                
                self.screen.blit(countdown_text, text_rect)
                
                ready_text = self.font_medium.render("¡Prepárate!", True, WHITE)
                self.screen.blit(ready_text, (WINDOW_WIDTH//2 - 100, WINDOW_HEIGHT//2 - 30))
            else:
                self.showing_countdown = False
        
        # Dibujar indicador de pose objetivo
        self.draw_pose_indicator()
    
    def draw_menu(self):
        self.screen.fill((30, 30, 50))
        
        title = self.font_large.render("HOLE IN THE WALL", True, YELLOW)
        self.screen.blit(title, (WINDOW_WIDTH//2 - 280, 80))
        
        subtitle = self.font_medium.render("Versión Simplificada", True, WHITE)
        self.screen.blit(subtitle, (WINDOW_WIDTH//2 - 180, 160))
        
        instructions = [
            "📷 Ponte frente a la cámara",
            "👋 Solo necesitas mover los BRAZOS",
            "⭐ 40% o más para pasar (¡muy fácil!)",
            "💡 Mira el indicador de pose a la derecha",
            "⏱️ Tienes 10 segundos por pose",
            "",
            "🎮 CONTROLES:",
            "ESPACIO - Comenzar",
            "ESC - Salir"
        ]
        
        for i, line in enumerate(instructions):
            text = self.font_small.render(line, True, WHITE)
            self.screen.blit(text, (WINDOW_WIDTH//2 - 280, 230 + i * 42))
        
        # Vista previa de cámara
        camera_surface = self.process_camera()
        if camera_surface:
            preview = pygame.transform.scale(camera_surface, (320, 240))
            self.screen.blit(preview, (WINDOW_WIDTH - 360, WINDOW_HEIGHT - 280))
            pygame.draw.rect(self.screen, WHITE, (WINDOW_WIDTH - 360, WINDOW_HEIGHT - 280, 320, 240), 2)
    
    def draw_gameover(self):
        self.screen.fill((50, 30, 30))
        
        title = self.font_large.render("GAME OVER", True, RED)
        self.screen.blit(title, (WINDOW_WIDTH//2 - 180, 150))
        
        score_text = self.font_medium.render(f"Puntuación Final: {self.score}", True, WHITE)
        self.screen.blit(score_text, (WINDOW_WIDTH//2 - 180, 260))
        
        level_text = self.font_medium.render(f"Nivel Alcanzado: {self.level}", True, WHITE)
        self.screen.blit(level_text, (WINDOW_WIDTH//2 - 160, 320))
        
        restart_text = self.font_small.render("ESPACIO - Reiniciar", True, YELLOW)
        self.screen.blit(restart_text, (WINDOW_WIDTH//2 - 130, 420))
        
        menu_text = self.font_small.render("ESC - Menú", True, WHITE)
        self.screen.blit(menu_text, (WINDOW_WIDTH//2 - 70, 470))
    
    def update(self):
        if self.game_state != "playing":
            return
        
        # No mover durante countdown
        if self.showing_countdown:
            return
        
        # Mover pared (muy lento)
        self.wall_position -= self.wall_speed
        
        # Verificar si terminó el tiempo
        elapsed = time.time() - self.wall_start_time
        if elapsed >= self.time_per_wall:
            match_score = self.get_smoothed_score()
            self.last_match_score = match_score
            self.show_result = True
            self.result_time = time.time()
            
            # UMBRAL MUY BAJO: 40%
            if match_score >= 40:
                bonus = int(match_score * 2)  # Doble de puntos
                self.score += bonus
                
                # Subir de nivel cada 500 puntos
                if self.score > self.level * 500:
                    self.level += 1
                    self.wall_speed = min(4, 1.5 + self.level * 0.3)
                    self.time_per_wall = max(7.0, 10.0 - self.level * 0.3)
            else:
                self.lives -= 1
                if self.lives <= 0:
                    self.game_state = "gameover"
                    return
            
            self.select_new_pose()
    
    def reset_game(self):
        self.score = 0
        self.lives = 3
        self.level = 1
        self.wall_speed = 1.5
        self.time_per_wall = 10.0
        self.select_new_pose()
        self.game_state = "playing"
    
    def run(self):
        running = True
        
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        if self.game_state == "playing":
                            self.game_state = "menu"
                        else:
                            running = False
                    elif event.key == pygame.K_SPACE:
                        if self.game_state == "menu":
                            self.reset_game()
                        elif self.game_state == "gameover":
                            self.game_state = "menu"
                    elif event.key == pygame.K_d:
                        # Debug: mostrar posición actual de brazos
                        if self.game_state == "playing":
                            left = self.check_arm_position(11, 13, 15)
                            right = self.check_arm_position(12, 14, 16)
                            print(f"DEBUG - Izq: {left}, Der: {right}, Score: {self.calculate_match_score()}%")
            
            # Procesar cámara
            camera_surface = self.process_camera()
            
            # Dibujar según estado
            if self.game_state == "menu":
                self.draw_menu()
            
            elif self.game_state == "playing":
                self.screen.fill((20, 20, 40))
                
                # Marco de cámara
                pygame.draw.rect(self.screen, WHITE, (48, 108, 484, 364), 3)
                
                self.draw_player_skeleton()
                self.draw_ui()
                self.update()
            
            elif self.game_state == "gameover":
                self.draw_gameover()
            
            pygame.display.flip()
            self.clock.tick(FPS)
        
        self.cap.release()
        self.pose.close()
        self.segmenter.close()
        pygame.quit()


if __name__ == "__main__":
    game = HoleInTheWallGame()
    game.run()

I0000 00:00:1764064610.495850  888229 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2
I0000 00:00:1764064610.497772  888229 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2
W0000 00:00:1764064610.500091  924142 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764064610.569472  924134 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764064610.583059  924136 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
